In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

### Count of numbers and lettice

In [ ]:
tr = pd.read_csv('../../data/train_groups.csv')
ts  = pd.read_csv('../../data/test_groups.csv')

test['is_train'] = False
train['is_train'] = True

title = pd.read_csv('information.csv')
data = data_x.merge(title, how = 'left', on = 'doc_id')
data = data.replace(np.nan, 'noinformation')

### Statistic

In [ ]:
numbers = []
num = np.arange(10).astype(str)
for i in data.title:
    numbers.append(sum(letter in num for letter in i))
    
    
data['numbers'] = numbers


vowels = []
vowel = ['а', 'о', 'и', 'е', 'ё', 'э', 'ы', 'у', 'ю', 'я']
for i in data.title:
    vowels.append(sum(letter in vowel for letter in i))
data['vowels'] = vowels


consonants = []
consonant = ['б', 'в', 'г', 'д', 'ж', 'з', 'й', 'к', 'л', 'м', 'н', 'п', 'р','с', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ']
for i in data.title:
    consonants.append(sum(letter in consonant for letter in i))
data['consonants'] = consonants

### Копипаст кода Севы

In [ ]:
doc_to_title = {}
with open('titles_norm_ru.csv', encoding='utf-8', newline='') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

In [ ]:
train_data = pd.read_csv('../../data/train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [ ]:
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:19]    )
X_train = np.column_stack((X_train, train_st[['numbers','vowels', 'consonants']]))
# X_train = np.column_stack((X_train, new_train[['mean_true_hash']]))
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

In [ ]:
test_data = pd.read_csv('../../data/test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [ ]:
X_test = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:19]    )
X_test = np.column_stack((X_test, test_st[['numbers','vowels', 'consonants']]))
# X_test = np.column_stack((X_test, new_test[['mean_true_hash']]))
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

### TFIDF 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
                             dtype = 'float32',
                        use_idf=True,
                        ngram_range=(1,3), # considering only 1-grams
) 

data_tf = vectorizer.fit_transform(data.information)

train_tf = data_tf[:len(train_data)]
test_tf = data_tf[len(train_data):]

In [ ]:
from scipy.sparse import hstack

train_new = hstack([train_tf, X_train])
test_new = hstack([test_tf, X_test])

### Make Cross val 

In [ ]:
groups = train_data.group_id.unique()
length = len(train_data[train_data['group_id'].isin(groups[:int(len(groups)*4/6)])])
X = train_new[:length]
X_val = train_new[length:]
y = y_train[:length]
y_val = y_train[length:]

In [5]:
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier 
from tqdm import tqdm

### Find best parametrs

In [ ]:


max_depth = [2, 3, 4, 5, 6, 7, 8]
learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1]
# random_state = [45, 100, 2019]


scores = []
params = []
parameters = product(max_depth, learning_rate)
parameters_list = list(parameters)

for param in tqdm(parameters_list, total = len(parameters_list)):
    xgb = XGBClassifier(max_depth = param[0], learning_rate = param[1], random_state = 45, n_estimators  = 500, n_jobs = -1)
                     clf__batches=param[2],
                     clf__model_type = 'log_reg')
    model.fit(X, y)
    y_pred = model.predict(X_val)
    scores.append(f1_score(y_val, y_pred))
    params.append((param[0], param[1], param[2]))
    print(f1_score(y_val, y_pred))

